# Nazwa i parametry workera

In [ ]:
import socket
import subprocess
import random
import os


hostname = socket.gethostname()
product_name = subprocess.check_output("nvidia-smi -q | grep 'Product Name' | cut -c45- | tr ' ' '_'", shell=True).decode("utf-8")[:-1]
worker_name = f"{hostname}-{product_name}-w{random.randint(1, 10)}"
print(worker_name, '\n')
os.environ['WORKER_NAME'] = worker_name

nvidia_smi = product_name = subprocess.check_output("nvidia-smi", shell=True).decode("utf-8")
print(nvidia_smi)

401c2e98e781-Tesla_P100-PCIE-16GB-w5 

Thu Jun  3 12:07:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                 

# Instalacja pakietów ClearML (uwaga: wymaga restartu i ponownego uruchomienia notebooka)

In [ ]:
!rm ./env_restart_required 2>/dev/null
!pip show clearml || (touch ./env_restart_required && pip install clearml numpy clearml-agent 'pyjwt<1.8.0')
!(ls -l ./env_restart_required 2>/dev/null && echo "Restart required.") || echo "Ready to go." 
import os
os.kill(os.getpid(), 9) if os.path.exists('./env_restart_required') else None

Name: clearml
Version: 1.0.3
Summary: ClearML - Auto-Magical Experiment Manager, Version Control, and MLOps for AI
Home-page: https://github.com/allegroai/clearml
Author: Allegroai
Author-email: clearml@allegro.ai
License: Apache License 2.0
Location: /usr/local/lib/python3.7/dist-packages
Requires: urllib3, pathlib2, six, Pillow, psutil, furl, pyparsing, PyYAML, future, attrs, requests, jsonschema, pyjwt, humanfriendly, python-dateutil, numpy
Required-by: 
Ready to go.


# Podpięcie dysku

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Sprawdzenie regionu w którym pracuje instancja (opcjonalne)

In [ ]:
!ls ./gcping_linux_amd64_latest || cp /content/drive/Shareddrives/RecSys21/colab/gcping_linux_amd64_latest .
!chmod +x ./gcping_linux_amd64_latest
!./gcping_linux_amd64_latest | head

ls: cannot access './gcping_linux_amd64_latest': No such file or directory
 1.  [asia-east1]               13.058671ms
 2.  [global]                   16.9059ms
 3.  [asia-east2]               159.812849ms
 4.  [asia-northeast2]          222.900584ms
 5.  [asia-northeast1]          225.095173ms
 6.  [asia-southeast1]          305.547076ms
 7.  [asia-northeast3]          350.411793ms
 8.  [asia-southeast2]          378.629513ms
 9.  [asia-south1]              652.829951ms
10.  [australia-southeast1]     846.412868ms


# Konfiguracja i uruchomienie workera



In [ ]:
%%bash

cp /content/drive/Shareddrives/RecSys21/colab/clearml.conf /root

export agent=`cat <<EOF
# Additional agent configuration
agent {
  worker_id: $WORKER_NAME
}
EOF
`
echo "$agent" >> /root/clearml.conf

In [ ]:
!clearml-agent daemon --queue default --foreground